In [5]:
import os
import pandas as pd
import numpy as np
import datetime
import time
import pyodbc
import calendar

start = time.time()

now = datetime.datetime.now()

week_name = input("Please enter the weekname in mm_dd_yy format or write EOM: ")

print(f"The process started at {now}")
folder_path = r"\\cernfs01\RevWx_BLR\BOS\Business Intelligence\Prodcutivity Reports\December_2020"+'\\'+week_name

def last_day_month(date):
    """Date in string format mm_dd_yy
    and get the last date in datetime format"""
    date = date.split("_")
    date[1] = str(1)
    mycal = calendar.monthcalendar(int(date[2]), int(date[0]))
    date[1] =  str(max(max(mycal)))
    activity_date_1 = "-".join(date)
    last_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
    return last_date




def weektoactivity(date):
    """Getting the activity date from the week_name input"""
    global activity_date
    date = date.split("_")
    if int(date[1]) + 7 < last_day_month("_".join(date)).day:
        date[1] = str(int(date[1])+7)
        activity_date_1 = "-".join(date)
        activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
        return activity_date
    else:
        if int(date[0]) == 12:
            num = last_day_month("_".join(date)).weekday()
            date[0] = "01"
            date[2] = str(int(date[2])+1)
            date[1] = str(5 - num)
            activity_date_1 = "-".join(date)
            activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
            print(activity_date)
            return activity_date
        else:
            num = last_day_month("_".join(date)).weekday()
            date[0] = str(int(date[0])+1)
            if num == 6:
                date[1] = "06"
                activity_date_1 = "-".join(date)
                activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
                return activity_date
            if num == 5:
                date[1] = "07"
                activity_date_1 = "-".join(date)
                activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
                return activity_date
                
            else:
                date[1] = str(5-num)
                activity_date_1 = "-".join(date)
                activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
                return activity_date
                
        
weektoactivity(week_name)

list_of_files = os.listdir(folder_path)


with_folder_files = []
for i in list_of_files:
    with_folder_file = os.path.join(folder_path,i)
    with_folder_files.append(with_folder_file)

    
    
all_data = pd.DataFrame()

for i in with_folder_files:
    df = pd.read_excel(i, sheet_name='Pivot')
    df.columns = df.loc[0]
    dff = df.copy()
    dff = dff.loc[1:]
    dff = dff.rename(columns={'DOS Aging Bucket':'Status'})
    dff.reset_index(inplace= True, drop=True)
    dff['Final Status'] = None
    for j in range(1,len(dff[np.NaN])):
        dff['Final Status'][j] =  dff[np.NaN][j]
    dff.drop(np.NaN, axis=1)
    dff = dff[1:]
    dff['Activity Date'] = activity_date
    dff['Client'] = (i.split(".")[0]).split("\\")[-1]
    dff.reset_index()
    all_data = all_data.append(dff, ignore_index=True)
    all_data = all_data.drop(np.NaN, axis=1)
print("All data uploaded to dataframe in python")

new_col_list = ['Final Status','Status','A. 0 to 30 days',
         'B. 31 to 60 days','C. 61 to 90 days','D. 91 to 120 days',
       'E. 121 to 150 days','F. 151 to 180 days','G. 181 to 365 days','H. 366 and above', 'Activity Date', 'Client']

all_data = all_data[new_col_list]
all_data = all_data.reset_index()
all_data = all_data.drop('index', axis=1)
#all_data.fillna(0)
all_data = all_data.to_numpy()

""" 
Starting the SQL update using pyodbc library. The values in the tables are 
converted to numpy arrays so that the same can be appended using for loop into desired SQL Server
"""


#Connecting to SQL Server
server_name = "W1751904\LOCAL_CERNER"
database_name = "Test_Productivity_Dashboard"
conn = pyodbc.connect(Driver='{SQL Server Native Client 11.0}',
                      Server=server_name,
                      Database=database_name,
                      trusted_connection='yes')


cursor = conn.cursor()


#Uploading in SQL Action Code table
insert_query = """INSERT INTO EATBxtract ([Final Status]
      ,[Status]
      ,[0 to 30 days]
      ,[31 to 60 days]
      ,[61 to 90 days]
      ,[91 to 120 days]
      ,[121 to 150 days]
      ,[151 to 180 days]
      ,[181 to 365 days]
      ,[366 above]
      ,[Activity Date]
      ,[Client]) 
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?)
                """


for row in all_data:
    values = (row[0],row[1],row[2],row[3],row[4],row[5],
              row[6],row[7],row[8],row[9],
              row[10],row[11])
    cursor.execute(insert_query,values)

conn.commit()
print("EATB Pivot uploaded to SQL Server")

    

end = time.time()    
print(f"The process completed at {round((end - start)/60)} minutes")   

        


Please enter the weekname in mm_dd_yy format or write EOM: 12_26_2020
The process started at 2021-01-06 15:00:28.092849
2021-01-02 00:00:00
All data uploaded to dataframe in python
EATB Pivot uploaded to SQL Server
The process completed at 1 minutes


In [111]:

def weektoactivity(week_name):
    
    global activity_date
    if week_name != 'EOM':
        week_name = week_name.split("_")
        activity_date_1 = "-".join(week_name)
        activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
    else:
        activity_date = last_day_of_month(datetime.date(now.year, now.month,1))
        
    
    

In [116]:
week_name = '12_19_2020'

In [127]:
dff

,NaN,Status,A. 0 to 30 days,B. 31 to 60 days,C. 61 to 90 days,D. 91 to 120 days,E. 121 to 150 days,F. 151 to 180 days,G. 181 to 365 days,H. 366 and above,Final Status,Activity Date,Client
1,Discharge 0-30,Non-Workable,5,0,0,0,0,0,0,0,Discharge 0-30,2020-12-31,WCDH_CO
2,FBD 0-30,Non-Workable,1379,169,5,1,0,0,2,0,FBD 0-30,2020-12-31,WCDH_CO
3,LBD 0-30,Non-Workable,23,201,56,30,22,9,16,4,LBD 0-30,2020-12-31,WCDH_CO
4,Not-Factored,Not-Factored,557,13,12,10,6,2,2,2,Not-Factored,2020-12-31,WCDH_CO
5,Self Pay/Zero Balance,Not-Factored,215,424,309,249,266,180,676,1046,Self Pay/Zero Balance,2020-12-31,WCDH_CO
6,Technical Denial,Workable,0,61,60,22,33,34,91,23,Technical Denial,2020-12-31,WCDH_CO
7,Workable,Workable,0,68,24,9,13,5,28,2,Workable,2020-12-31,WCDH_CO
8,Worked in last 15 days,Non-Workable,0,53,23,15,3,6,8,1,Worked in last 15 days,2020-12-31,WCDH_CO


In [118]:
def last_day_of_month(any_day):
    """Getting the last day of the month"""
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - datetime.timedelta(days=next_month.day)

In [122]:
def last_day_of_month(any_day):
    """Getting the last day of the month"""
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - datetime.timedelta(days=next_month.day)

def weektoactivity(week_name):
    """to get the activity date for the table"""
    global activity_date
    if week_name != 'EOM':
        week_name = week_name.split("_")
        activity_date_1 = "-".join(week_name)
        activity_date = datetime.datetime.strptime(activity_date_1, "%m-%d-%Y")
    else:
        activity_date = last_day_of_month(datetime.date(now.year,12,1))

In [128]:
week_name = '12_26_2020'

In [129]:
weektoactivity(week_name)

In [130]:
activity_date

datetime.datetime(2020, 12, 26, 0, 0)

In [39]:
new_col_list = ['Final Status','Status','A. 0 to 30 days',
         'B. 31 to 60 days','C. 61 to 90 days','D. 91 to 120 days',
       'E. 121 to 150 days','F. 151 to 180 days','G. 181 to 365 days','H. 366 and above', 'Activity Date', 'Client']

In [69]:
dff['Final Status'] = None

for i in range(1,len(dff[np.NaN])):
    dff['Final Status'][i] =  dff[np.NaN][i]

    
    

C:\Users\GN082282\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [73]:
dff = dff.drop([np.NaN], axis = 1)

In [76]:
dff = dff.rename(columns={'DOS Aging Bucket':'Status'})

In [80]:
dff.reset_index(inplace= True, drop=True)

In [83]:
dff = dff[1:]

In [10]:
all_data


,Status,A. 0 to 30 days,B. 31 to 60 days,C. 61 to 90 days,D. 91 to 120 days,E. 121 to 150 days,F. 151 to 180 days,G. 181 to 365 days,H. 366 and above,Final Status,Activity Date,Client
0,Non-Workable,40,0,0,0,0,0,0,0,Discharge 0-30,2020-12-19,Barnabas
1,Non-Workable,44553,7678,2142,861,571,224,453,74,FBD 0-30,2020-12-19,Barnabas
2,Non-Workable,359,4706,2814,1387,1000,495,1321,220,LBD 0-30,2020-12-19,Barnabas
3,Not-Factored,29169,6175,3692,2041,1363,746,3463,1042,Not-Factored,2020-12-19,Barnabas
4,Not-Factored,5898,16332,13635,11968,11222,7660,26341,19379,Self Pay/Zero Balance,2020-12-19,Barnabas
...,...,...,...,...,...,...,...,...,...,...,...,...
132,Not-Factored,1178,33,91,10,8,1,39,232,Not-Factored,2020-12-19,South_Florida
133,Not-Factored,253,535,578,451,444,333,1181,3733,Self Pay/Zero Balance,2020-12-19,South_Florida
134,Workable,0,323,254,205,240,167,964,2749,Technical Denial,2020-12-19,South_Florida
135,Workable,0,308,347,339,227,78,442,185,Workable,2020-12-19,South_Florida


In [11]:
j

7

In [12]:
i

'\\\\cernfs01\\RevWx_BLR\\BOS\\Business Intelligence\\Prodcutivity Reports\\December_2020\\12_19_2020\\TORR_CA.xlsx'

In [18]:
all_data

,NaN,Status,A. 0 to 30 days,B. 31 to 60 days,C. 61 to 90 days,D. 91 to 120 days,E. 121 to 150 days,F. 151 to 180 days,G. 181 to 365 days,H. 366 and above,Final Status,Activity Date,Client
0,Discharge 0-30,Non-Workable,14,0,0,0,0,0,0,0,Discharge 0-30,2020-12-19,CH_CA
1,FBD 0-30,Non-Workable,6168,1360,11,6,1,0,0,0,FBD 0-30,2020-12-19,CH_CA
2,LBD 0-30,Non-Workable,32,190,86,66,31,21,45,17,LBD 0-30,2020-12-19,CH_CA
3,Not-Factored,Not-Factored,2563,80,27,18,6,9,33,83,Not-Factored,2020-12-19,CH_CA
4,Self Pay/Zero Balance,Not-Factored,93,394,335,281,284,253,1094,2297,Self Pay/Zero Balance,2020-12-19,CH_CA
5,Technical Denial,Workable,0,185,220,211,289,257,457,258,Technical Denial,2020-12-19,CH_CA
6,Workable,Workable,0,1870,1040,570,181,63,117,240,Workable,2020-12-19,CH_CA
7,Worked in last 15 days,Non-Workable,0,584,527,440,116,8,8,0,Worked in last 15 days,2020-12-19,CH_CA
8,Discharge 0-30,Non-Workable,642,0,0,0,0,0,0,0,Discharge 0-30,2020-12-19,CH_CA
9,FBD 0-30,Non-Workable,19465,1484,51,28,16,6,10,9,FBD 0-30,2020-12-19,CH_CA


In [2]:
import datetime


now = datetime.datetime.now()

In [3]:
datetime.datetime(now.year, now.month, 19)

datetime.datetime(2021, 1, 19, 0, 0)

In [7]:
import datetime
print(datetime.datetime.now())
print("Hello")

2021-01-06 17:15:38.657810
Hello
